# Get CF problem texts

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import psycopg2
import pandas as pd
import os.path

## Functions

### url request

In [2]:
from time import sleep
def getproblemtext(contest=None, problem=None, url=None):
    maxtries = 5
    tries = 0
    if url==None:
        assert(contest is not None and problem is not None)
        url = 'http://codeforces.com/problemset/problem/' + str(contest) + '/' + str(problem)
    print url
    
    while tries < maxtries:
        try:
            r = requests.get(url)
            return r.content
        except:
            tries += 1
            sleep(5)
    print "ERROR GETTING CONTEST INFO FOR", contest, problem_data.csv

### scientific notation -> 0's

In [3]:
# turn scientific notation into 0s
def sci2num(txt):
    #txt = txt.encode('utf-8')
    exponents = map(int, re.findall('10<sup class="upper-index">(.)</sup>', txt))
    for e in exponents:
        estr = '10' + ''.join(map(str, [0]*(e-1)))
        txt = re.sub('10<sup class="upper-index">' + str(e) + '</sup>', estr, txt)
    #return BeautifulSoup(txt, 'html.parser')
    return txt



### Ignore first tag

In [4]:
def ignore1sttag(bs4obj):
    inputlist = []
    for cont in bs4obj.contents:
        if type(cont) == bs4.NavigableString:
            inputlist.append(cont)
        elif len(cont.attrs) == 0:
            inputlist.append(cont.text)
    out = ''.join(inputlist)
    return out

### parsing

In [5]:
def get_problem_dict(contestid, problemid, url=None):
    if url is None:
        r = getproblemtext(contestid, problemid)
    else:
        r = getproblemtext(url=url)
    r = sci2num(r)

    soup = BeautifulSoup(r, 'html.parser')
    #print(soup.prettify())

    statement = soup.find("div", { "class" : "problem-statement" })
    problem_name = soup.find("div", { "class" : "title" }).string

    tmp = soup.find("div", { "class" : "time-limit" })
    timelimit = tmp.contents[-1]
    timelimit = timelimit.split(' ')[0]
    timelimit = float(timelimit)

    tmp = soup.find("div", { "class" : "memory-limit" })
    memlimit = tmp.contents[-1]
    memlimit = memlimit.split(' ')[0]
    memlimit = int(memlimit)

    children = []
    for c in statement.children:
        children.append(c)

    description = children[1]
    assert(len(description.attrs) == 0)
    txt_descr = description.text

    txt_input = ignore1sttag(children[2])
        
    txt_output = ignore1sttag(children[3])
    
    if (len(children) >= 6):
        txt_note = ignore1sttag(children[5])
    else:
        txt_note = ''

    assert(children[4].attrs['class'][0] == u'sample-tests')

    #print problem_name, '\n'
    #print timelimit, '\n'
    #print memlimit, '\n'
    #print txt_descr, '\n'
    #print txt_input, '\n'
    #print txt_output, '\n'
    #print txt_note, '\n'

    problem_dict = {
        'contestid':     contestid,
        'problemid':     problemid,
        'problemname':   problem_name,
        'timelimit' :    timelimit,
        'memlimit' :     memlimit,
        'txt_descr' :    txt_descr,
        'txt_input' :    txt_input,
        'txt_output':    txt_output,
        'txt_note':      txt_note
    }

    return problem_dict

## Calling functions

In [327]:
db = 'codeforces'
usr = 'Joy'
con = psycopg2.connect(database = db, user = usr)
cur = con.cursor()

In [358]:
query = """
SELECT contestid, problemid FROM problem_info;
"""
cur.execute(query)
cid_pid = cur.fetchall()
#df_cid_pid = pd.read_sql(query, con)

### number of problems in database
* total: 4,248
* have some tags: 3,021

In [334]:
df_cid_pid.shape

(4248, 2)

In [342]:
con.rollback()

query = """
SELECT COUNT(*) FROM (SELECT DISTINCT contestid, problemid FROM tags) AS temp;
"""
cur.execute(query)
cur.fetchall()[0][0]

3021L

### get question texts from CF

In [391]:
txtlist = []
idx = 0
last_idx = 874

out_file = 'problem_texts.tsv'

for cid, pid in cid_pid[last_idx:]:
    print last_idx + idx
    try:
        txtlist.append(get_problem_dict(cid, pid))
    except:
        with open('CF_problem_skipped.txt', 'a') as f:
            print "SKIPPED", cid, pid
            f.write(str(cid) + "," + str(pid) + "\n")
    
    if idx%5==0:
        df_out = pd.DataFrame.from_dict(txtlist)
        if (os.path.exists(out_file)):
            df_out.to_csv(out_file, sep='\t', header=False, index=False, encoding='utf-8', mode='a')
        else:
            df_out.to_csv(out_file, sep='\t', header=True, index=False, encoding='utf-8', mode='w')
        txtlist = []
        print "------------------------- written to file", out_file
        
    idx+=1;
#get_problem_dict(contestid, problemid)

874
http://codeforces.com/problemset/problem/208/E
------------------------- written to file problem_texts.tsv
875
http://codeforces.com/problemset/problem/211/A
SKIPPED 211 A
876
http://codeforces.com/problemset/problem/211/B
SKIPPED 211 B
877
http://codeforces.com/problemset/problem/211/C
SKIPPED 211 C
878
http://codeforces.com/problemset/problem/211/D
SKIPPED 211 D
879
http://codeforces.com/problemset/problem/211/E
SKIPPED 211 E
------------------------- written to file problem_texts.tsv
880
http://codeforces.com/problemset/problem/213/A
881
http://codeforces.com/problemset/problem/213/B
882
http://codeforces.com/problemset/problem/213/C
 Furik's path is marked with yellow, and Rubik's path is marked with pink.
883
http://codeforces.com/problemset/problem/213/D
884
http://codeforces.com/problemset/problem/213/E
------------------------- written to file problem_texts.tsv
885
http://codeforces.com/problemset/problem/214/A
886
http://codeforces.com/problemset/problem/214/B
887
http://c

989
http://codeforces.com/problemset/problem/234/E
------------------------- written to file problem_texts.tsv
990
http://codeforces.com/problemset/problem/234/F
991
http://codeforces.com/problemset/problem/234/G
992
http://codeforces.com/problemset/problem/234/H
993
http://codeforces.com/problemset/problem/235/A
994
http://codeforces.com/problemset/problem/235/B
------------------------- written to file problem_texts.tsv
995
http://codeforces.com/problemset/problem/235/C
996
http://codeforces.com/problemset/problem/235/D
997
http://codeforces.com/problemset/problem/235/E
998
http://codeforces.com/problemset/problem/236/A
999
http://codeforces.com/problemset/problem/236/B
------------------------- written to file problem_texts.tsv
1000
http://codeforces.com/problemset/problem/236/C
1001
http://codeforces.com/problemset/problem/236/D
1002
http://codeforces.com/problemset/problem/236/E
1003
http://codeforces.com/problemset/problem/237/A
1004
http://codeforces.com/problemset/problem/237/B

1116
http://codeforces.com/problemset/problem/261/C
1117
http://codeforces.com/problemset/problem/261/D
1118
http://codeforces.com/problemset/problem/261/E
1119
http://codeforces.com/problemset/problem/262/A
------------------------- written to file problem_texts.tsv
1120
http://codeforces.com/problemset/problem/262/B
1121
http://codeforces.com/problemset/problem/262/C
1122
http://codeforces.com/problemset/problem/262/D
1123
http://codeforces.com/problemset/problem/262/E
1124
http://codeforces.com/problemset/problem/263/A
------------------------- written to file problem_texts.tsv
1125
http://codeforces.com/problemset/problem/263/B
1126
http://codeforces.com/problemset/problem/263/C
1127
http://codeforces.com/problemset/problem/263/D
1128
http://codeforces.com/problemset/problem/263/E
1129
http://codeforces.com/problemset/problem/264/A
------------------------- written to file problem_texts.tsv
1130
http://codeforces.com/problemset/problem/264/B
1131
http://codeforces.com/problemset/pr

------------------------- written to file problem_texts.tsv
1245
http://codeforces.com/problemset/problem/288/B
1246
http://codeforces.com/problemset/problem/288/C
1247
http://codeforces.com/problemset/problem/288/D
1248
http://codeforces.com/problemset/problem/288/E
1249
http://codeforces.com/problemset/problem/289/A
------------------------- written to file problem_texts.tsv
1250
http://codeforces.com/problemset/problem/289/B
1251
http://codeforces.com/problemset/problem/289/C
1252
http://codeforces.com/problemset/problem/289/D
1253
http://codeforces.com/problemset/problem/289/E
1254
http://codeforces.com/problemset/problem/292/A
------------------------- written to file problem_texts.tsv
1255
http://codeforces.com/problemset/problem/292/B
1256
http://codeforces.com/problemset/problem/292/C
1257
http://codeforces.com/problemset/problem/292/D
1258
http://codeforces.com/problemset/problem/292/E
1259
http://codeforces.com/problemset/problem/293/A
------------------------- written to fil

1371
http://codeforces.com/problemset/problem/319/E
1372
http://codeforces.com/problemset/problem/320/A
1373
http://codeforces.com/problemset/problem/320/B
1374
http://codeforces.com/problemset/problem/320/C
------------------------- written to file problem_texts.tsv
1375
http://codeforces.com/problemset/problem/320/D
1376
http://codeforces.com/problemset/problem/320/E
1377
http://codeforces.com/problemset/problem/321/A
1378
http://codeforces.com/problemset/problem/321/B
1379
http://codeforces.com/problemset/problem/321/C
------------------------- written to file problem_texts.tsv
1380
http://codeforces.com/problemset/problem/321/D
1381
http://codeforces.com/problemset/problem/321/E
1382
http://codeforces.com/problemset/problem/322/A
1383
http://codeforces.com/problemset/problem/322/B
1384
http://codeforces.com/problemset/problem/322/C
------------------------- written to file problem_texts.tsv
1385
http://codeforces.com/problemset/problem/322/D
1386
http://codeforces.com/problemset/pr

...

3721
http://codeforces.com/problemset/problem/575/G
3722
http://codeforces.com/problemset/problem/575/H
3723
http://codeforces.com/problemset/problem/575/I
3724
http://codeforces.com/problemset/problem/566/A
------------------------- written to file problem_texts.tsv
3725
http://codeforces.com/problemset/problem/566/B
3726
http://codeforces.com/problemset/problem/566/C
3727
http://codeforces.com/problemset/problem/566/D
3728
http://codeforces.com/problemset/problem/566/E
3729
http://codeforces.com/problemset/problem/566/F
------------------------- written to file problem_texts.tsv
3730
http://codeforces.com/problemset/problem/566/G
3731
http://codeforces.com/problemset/problem/562/A
SKIPPED 562 A
3732
http://codeforces.com/problemset/problem/562/B
SKIPPED 562 B
3733
http://codeforces.com/problemset/problem/562/C
SKIPPED 562 C
3734
http://codeforces.com/problemset/problem/562/D
SKIPPED 562 D
------------------------- written to file problem_texts.tsv
3735
http://codeforces.com/problemse

------------------------- written to file problem_texts.tsv
3845
http://codeforces.com/problemset/problem/397/E
3846
http://codeforces.com/problemset/problem/394/A
3847
http://codeforces.com/problemset/problem/394/B
3848
http://codeforces.com/problemset/problem/394/C
3849
http://codeforces.com/problemset/problem/394/D
------------------------- written to file problem_texts.tsv
3850
http://codeforces.com/problemset/problem/394/E
3851
http://codeforces.com/problemset/problem/392/A
3852
http://codeforces.com/problemset/problem/392/B
3853
http://codeforces.com/problemset/problem/392/C
3854
http://codeforces.com/problemset/problem/392/D
------------------------- written to file problem_texts.tsv
3855
http://codeforces.com/problemset/problem/392/E
3856
http://codeforces.com/problemset/problem/393/A
3857
http://codeforces.com/problemset/problem/393/B
3858
http://codeforces.com/problemset/problem/393/C
3859
http://codeforces.com/problemset/problem/393/D
------------------------- written to fil

3973
http://codeforces.com/problemset/problem/207/A3
3974
http://codeforces.com/problemset/problem/207/B1
------------------------- written to file problem_texts.tsv
3975
http://codeforces.com/problemset/problem/207/B2
3976
http://codeforces.com/problemset/problem/207/B3
3977
http://codeforces.com/problemset/problem/207/C1
3978
http://codeforces.com/problemset/problem/207/C2
3979
http://codeforces.com/problemset/problem/207/C3
------------------------- written to file problem_texts.tsv
3980
http://codeforces.com/problemset/problem/207/D1
3981
http://codeforces.com/problemset/problem/207/D10
3982
http://codeforces.com/problemset/problem/207/D2
3983
http://codeforces.com/problemset/problem/207/D3
3984
http://codeforces.com/problemset/problem/207/D4
------------------------- written to file problem_texts.tsv
3985
http://codeforces.com/problemset/problem/207/D5
3986
http://codeforces.com/problemset/problem/207/D6
3987
http://codeforces.com/problemset/problem/207/D7
3988
http://codeforces.c

------------------------- written to file problem_texts.tsv
4095
http://codeforces.com/problemset/problem/125/E
4096
http://codeforces.com/problemset/problem/120/A
4097
http://codeforces.com/problemset/problem/120/B
4098
http://codeforces.com/problemset/problem/120/C
4099
http://codeforces.com/problemset/problem/120/D
------------------------- written to file problem_texts.tsv
4100
http://codeforces.com/problemset/problem/120/E
4101
http://codeforces.com/problemset/problem/120/F
4102
http://codeforces.com/problemset/problem/120/G
4103
http://codeforces.com/problemset/problem/120/H
4104
http://codeforces.com/problemset/problem/120/I
------------------------- written to file problem_texts.tsv
4105
http://codeforces.com/problemset/problem/120/J
4106
http://codeforces.com/problemset/problem/119/A
4107
http://codeforces.com/problemset/problem/119/B
4108
http://codeforces.com/problemset/problem/119/C
4109
http://codeforces.com/problemset/problem/119/D
------------------------- written to fil

------------------------- written to file problem_texts.tsv
4225
http://codeforces.com/problemset/problem/812/E
4226
http://codeforces.com/problemset/problem/811/A
4227
http://codeforces.com/problemset/problem/811/B
4228
http://codeforces.com/problemset/problem/811/C
4229
http://codeforces.com/problemset/problem/811/D
 
 
------------------------- written to file problem_texts.tsv
4230
http://codeforces.com/problemset/problem/811/E
4231
http://codeforces.com/problemset/problem/809/A
4232
http://codeforces.com/problemset/problem/809/B
SKIPPED 809 B
4233
http://codeforces.com/problemset/problem/809/C
4234
http://codeforces.com/problemset/problem/809/D
------------------------- written to file problem_texts.tsv
4235
http://codeforces.com/problemset/problem/809/E
4236
http://codeforces.com/problemset/problem/810/A
4237
http://codeforces.com/problemset/problem/810/B
4238
http://codeforces.com/problemset/problem/810/C
4239
http://codeforces.com/problemset/problem/810/D
SKIPPED 810 D
--------

### Redo errors

In [6]:
# CF_problems_redo.txt
with open('CF_problem_skipped.txt') as f:
    skipped = f.readlines()
skipped = [s.strip().split(',') for s in skipped]
skipped = skipped[1:]
newurl = "http://codeforces.com/contest/206/problem/A1"

In [8]:
txtlist = []
idx = 0
last_idx = 36

out_file = 'problem_texts.tsv'

for cid, pid in skipped[last_idx:]:
    print last_idx + idx
    url = "http://codeforces.com/contest/" + str(cid) + "/problem/" + str(pid)
    try:
        txtlist.append(get_problem_dict(cid, pid, url=url))
    except:
        with open('CF_problem_skipped2.txt', 'a') as f:
            print "SKIPPED", cid, pid
            f.write(str(cid) + "," + str(pid) + "\n")
    
    if idx%5==0:
        df_out = pd.DataFrame.from_dict(txtlist)
        if (os.path.exists(out_file)):
            df_out.to_csv(out_file, sep='\t', header=False, index=False, encoding='utf-8', mode='a')
        else:
            df_out.to_csv(out_file, sep='\t', header=True, index=False, encoding='utf-8', mode='w')
        txtlist = []
        print "------------------------- written to file", out_file
        
    idx+=1;

36
http://codeforces.com/contest/419/problem/D
------------------------- written to file problem_texts.tsv
37
http://codeforces.com/contest/419/problem/E
38
http://codeforces.com/contest/648/problem/A
SKIPPED 648 A
39
http://codeforces.com/contest/648/problem/B
SKIPPED 648 B
40
http://codeforces.com/contest/648/problem/C
SKIPPED 648 C
41
http://codeforces.com/contest/648/problem/D
SKIPPED 648 D
------------------------- written to file problem_texts.tsv
42
http://codeforces.com/contest/648/problem/E
SKIPPED 648 E
43
http://codeforces.com/contest/649/problem/A
SKIPPED 649 A
44
http://codeforces.com/contest/649/problem/B
SKIPPED 649 B
45
http://codeforces.com/contest/649/problem/C
SKIPPED 649 C
46
http://codeforces.com/contest/649/problem/D
SKIPPED 649 D
------------------------- written to file problem_texts.tsv
47
http://codeforces.com/contest/649/problem/E
SKIPPED 649 E
48
http://codeforces.com/contest/690/problem/E1
SKIPPED 690 E1
49
http://codeforces.com/contest/690/problem/E2
SKIPP

No handlers could be found for logger "bs4.dammit"


SKIPPED 636 A
------------------------- written to file problem_texts.tsv
82
http://codeforces.com/contest/562/problem/A
83
http://codeforces.com/contest/562/problem/B
84
http://codeforces.com/contest/562/problem/C
85
http://codeforces.com/contest/562/problem/D
86
http://codeforces.com/contest/562/problem/E
------------------------- written to file problem_texts.tsv
87
http://codeforces.com/contest/562/problem/F
88
http://codeforces.com/contest/562/problem/G
89
http://codeforces.com/contest/541/problem/A
90
http://codeforces.com/contest/541/problem/B
91
http://codeforces.com/contest/541/problem/C
------------------------- written to file problem_texts.tsv
92
http://codeforces.com/contest/541/problem/D
93
http://codeforces.com/contest/541/problem/E
94
http://codeforces.com/contest/541/problem/F
95
http://codeforces.com/contest/537/problem/A
96
http://codeforces.com/contest/532/problem/A
------------------------- written to file problem_texts.tsv
97
http://codeforces.com/contest/532/prob